## Pydantic OutputParser
Pydantic OutputParser란?
- LLM의 출력을 구조화된 데이터 형태로 변환해주는 도구입니다
- LLM이 생성한 텍스트를 Python 객체로 자동 파싱합니다
- 데이터 검증과 타입 안정성을 보장합니다
- JSON 형태의 응답을 Pydantic 모델로 변환하여 일관된 데이터 구조를 제공합니다
- 예시: 군인 정보를 이름, 계급, 부대 등의 구조화된 형태로 추출합니다

LLM 모델 객체 생성하기


In [ ]:
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model="midm-2.0-base-instruct-q5_k_m",
    temperature=0.1,  # 구조화된 출력을 위해 낮은 temperature 설정
)

Pydantic OutputParser 를 사용하지 않는 예시

In [ ]:
from langchain_core.prompts import PromptTemplate

# 프롬프트 템플릿 정의 (format_instructions 포함)
template = """
당신은 군대 정보 분석 전문가입니다.
주어진 텍스트에서 군인의 정보를 추출하여 구조화된 형태로 변환해주세요.

입력 텍스트: {text}
"""

# PromptTemplate 생성
prompt = PromptTemplate(
    template=template,
    input_variables=["text"],
)

# 프롬프트 미리보기
sample_text = "김철수 병장은 제1보병사단 소속이며, 보병 특기로 20개월째 복무중입니다."

formatted_prompt = prompt.format(text=sample_text)

print(formatted_prompt)


In [ ]:
chain = prompt | llm

chain.invoke(formatted_prompt).content

이러한 경우처럼 군인의 정보가 주어졌을때, 정보안에서 추출하고싶은 데이터들이 있다면,   
Pydantic 모델 형태로 정의된 클래스로 형태로 파싱하여 응답을 받을 수 있습니다.

1. Pydantic 모델 정의하기

In [ ]:
from pydantic import BaseModel, Field

class SoldierInfo(BaseModel):
    """군인 정보를 담는 Pydantic 모델"""
    name: str = Field(description="군인의 이름")
    rank: str = Field(description="계급 (예: 이병, 일병, 상병, 병장)")
    unit: str = Field(description="소속 부대")
    specialty: str = Field(description="특기 또는 병과")
    service_period: str = Field(description="복무 기간")

# 모델 출력 확인
print("SoldierInfo 모델 정의 완료")
print("모델 필드:", SoldierInfo.model_fields)


### `get_format_instructions()` 메서드
- Pydantic 모델의 스키마를 LLM이 이해할 수 있는 명확한 지침 문자열로 변환한다.  
- 변환된 포맷 지침을 기반으로, LLM이 응답을 Pydantic 모델 구조에 맞춰 생성하도록 도와준다.

In [ ]:
from langchain.output_parsers import PydanticOutputParser

# Pydantic OutputParser 생성
parser = PydanticOutputParser(pydantic_object=SoldierInfo)

# 파서가 어떤 형식을 요구하는지 확인
print(parser.get_format_instructions())


prompt 재정의
- `text` : 군인 정보가 들어있는 text (사용자 입력을 받을 위치)
- `format_instructions` : parser로 형식 지정

In [ ]:
from langchain_core.prompts import PromptTemplate

# 프롬프트 템플릿 정의 (format_instructions 포함)
template = """
당신은 군대 정보 분석 전문가입니다.
주어진 텍스트에서 군인의 정보를 추출하여 구조화된 형태로 변환해주세요.

입력 텍스트: {text}

{format_instructions}
"""

# PromptTemplate 생성
prompt = PromptTemplate(
    template=template,
    input_variables=["text"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

# 프롬프트 미리보기
print(formatted_prompt)


LCEL 체인 만들기
- 체인 구조: PromptTemplate → ChatOllama

In [ ]:
# LCEL을 사용한 체인 생성: 프롬프트 → LLM → 파서
chain = prompt | llm 

# 테스트 데이터
test_text = "이동훈 상병은 제3기계화보병사단 소속으로 정비병 특기를 가지고 있으며, 현재 18개월째 복무 중입니다."

# 체인 실행
result = chain.invoke({"text": test_text})

print(result.content)

Prompt에 작성된 것 처럼 json 형태로 응답이 오는것을 확인 할 수 있다.  
이 응답을 `parser` 를 사용해서 `SordierInfo` 객체로 변환해보자

`parser()` : LLM의 응답을 파싱하여 정의된 Pydantic 모델의 객체로 변환하는 함수
   - 입력: LLM의 JSON 형식 응답 문자열
   - 출력: Pydantic 모델의 객체로 변환되는 함수

In [ ]:
parsed_result = parser.parse(result.content);

print(parsed_result)

print(parsed_result.name) # 객체로 값들을 사용 할 수 있다.

In [ ]:
chain = prompt | llm | parser

test_text = "이동훈 상병은 제3기계화보병사단 소속으로 정비병 특기를 가지고 있으며, 현재 18개월째 복무 중입니다."


result = chain.invoke({"text": test_text})

print(result)

print(result.name)